In [ ]:
import os
import sys
import itertools
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno
import csv

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import Ridge, Lasso, LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
)

In [ ]:
path = "/Users/jrock/Documents/Yonsei/DSL/EDA/cp/onehotencode.csv"
data = pd.read_csv(path)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = data.drop(['#종속변수'], axis = 1)
y = data['#종속변수']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, shuffle=True, random_state=1)

mRMR

In [ ]:
k = 15
S = []


# Calculate the Spearman correlation matrix
corr_matrix = data.corr(method='spearman')

# Remove the target variable from the correlation matrix
corr_matrix = corr_matrix.drop('Annual Revenue LTM (USD)', axis=0)

# Find the feature with the highest relevance (minimum redundancy)
highest_corr_idx = np.argmin(np.abs(corr_matrix['Annual Revenue LTM (USD)']))
first_feature = corr_matrix.index[highest_corr_idx]

S.append(first_feature)

for i in range(k - 1):
    relevance = np.abs(corr_matrix['Annual Revenue LTM (USD)'])
    redundancy = np.mean(np.abs(corr_matrix.loc[:, S]), axis=1)
    candidate_feature = relevance - redundancy
    candidate_feature = candidate_feature.drop(S)
    best_x_idx = np.argmax(candidate_feature)
    next_feature = candidate_feature.index[best_x_idx]
    S.append(next_feature)

In [ ]:
S

In [ ]:
mrmr_features = S.copy()

In [ ]:
mrmr_X_train = X_train[mrmr_features]
mrmr_X_test = X_test[mrmr_features]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Create a RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=486)

# Train the regressor
rf_regressor.fit(mrmr_X_train, y_train)

# Assuming mrmr_X_test is your feature matrix for testing
# Use the trained model to make predictions on new data
y_pred = rf_regressor.predict(mrmr_X_test)

# Evaluate the performance of the model (you may use other metrics depending on your problem)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
rf = RandomForestRegressor(random_state=486)
rf.fit(mrmr_X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Use the trained model to make predictions on new data
test_pred = rf_regressor.predict(mrmr_X_test)

# Evaluate the performance of the regression model
mae = mean_absolute_error(y_test, test_pred)
mse = mean_squared_error(y_test, test_pred)
r2 = r2_score(y_test, test_pred)

print('[mRMR 변수 선택]')
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared (R2):", r2)

변수 중요도 출력

In [ ]:
feature_importances = rf_regressor.feature_importances_

feature_names = mrmr_X_train.columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or display the sorted feature importances
print('[mRMR 변수 선택]')
print(feature_importance_df)